In [1]:
from keras.models import load_model

import numpy as np
import librosa
import librosa.display
import sklearn
import matplotlib.pyplot as plt
import speech_recognition as VTT
from unicodedata import normalize
import os
import sys
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [3]:
VTE_Array = []
TTE_Array = []

In [5]:
CNN_model = load_model('weight.h5')

In [7]:
for filename in os.listdir("WAV/"):
    filename = normalize('NFC', filename)
    try:
        if '.wav' not in filename in filename:
            continue
        wav, sr = librosa.load("WAV/" + filename, sr=None)
        print(filename)
        
        mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=100, n_fft=400, hop_length=160)
        mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
        padded_mfcc = pad2d(mfcc, 700)
        padded_mfcc = np.expand_dims(padded_mfcc, 0)
        
        VTE_result = CNN_model.predict(padded_mfcc)
        VTE_Array.append(VTE_result)
    
    except Exception as e:
        print(filename, e)
        raise

Sample1.wav
1/1 [==============================] - 0s 72ms/step


C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


Sample2.wav
1/1 [==============================] - 0s 72ms/step


C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [8]:
if not os.path.exists('TXT'):
    os.makedirs('TXT')

for filename in os.listdir("WAV/"):
    if not filename.endswith('.wav'):
        continue
    
    filename = normalize('NFC', filename)
    try:
        r = VTT.Recognizer()
        kr_audio = VTT.AudioFile("WAV/" + filename)
        with kr_audio as source:
            VTT_audio = r.record(source)
            
        temp = r.recognize_google(VTT_audio, language='ko-KR')
        
        # Get the file name without extension
        name = os.path.splitext(filename)[0]
        with open("TXT/" + str(name) + 'out.txt', 'w') as f:
            first_line = temp.split('\n')[0]
            print(first_line, file=f)
            
    except Exception as e:
        print(f"Error processing file {filename}: {str(e)}")

result2:
{   'alternative': [   {   'confidence': 0.82105637,
                           'transcript': '얼굴은 화끈 화끈 하고 가슴이 무서워 하는 사람처럼 뛰어 '
                                         '놀았습니다'},
                       {'transcript': '얼굴은 화끈 화끈 하고 가슴이 무서워하는 사람처럼 뛰어 놀았습니다'},
                       {'transcript': '얼굴은 화끈화끈 하고 가슴이 무서워 하는 사람처럼 뛰어 놀았습니다'},
                       {'transcript': '얼굴은 화끈화끈 하고 가슴이 무서워하는 사람처럼 뛰어 놀았습니다'},
                       {'transcript': '얼굴은 화끈 화끈 하고 가슴이 무서워 하는 사람처럼 띠 놀았습니다'}],
    'final': True}
result2:
{   'alternative': [   {   'confidence': 0.8971411,
                           'transcript': '엿장수는 숟가락과 여탄 주먹을 바꿔주었다'},
                       {'transcript': '엿장수는 숟가락과 여탕 주먹을 바꿔주었다'},
                       {'transcript': '엿장수는 숟가락과 여타 주먹을 바꿔주었다'},
                       {'transcript': '엿장수는 숟가락과 연탄 주먹을 바꿔주었다'},
                       {'transcript': '엿장수는 숟가락과 여탐 주먹을 바꿔주었다'}],
    'final': True}


In [18]:
rnn_model = tf.keras.models.load_model("./WtoE", custom_objects={"TextVectorization":TextVectorization})

In [22]:
for filename in os.listdir("TXT/"):
    filename = normalize('NFC', filename)
    try:
        if '.txt' not in filename in filename:
            continue
        f = open('TXT/' + filename)
        input_txt = f.read().strip()
        f.close()
        
        emotion = float(rnn_model.predict([input_txt]))
        TTE_Array.append(emotion)
        
    except Exception as e:
        print(filename, e)
        raise

1/1 [==============================] - 0s 72ms/step


In [23]:
print(TTE_Array)

[array([[0.744904]], dtype=float32), array([[0.744904]], dtype=float32), array([[0.744904]], dtype=float32), array([[0.744904]], dtype=float32), 0.011249206028878689, 0.5051031708717346]
